# Data

Features per Team by Season per match
- F9
- goals scored till now


InputData per Event in match
- F9 features
- sum(event features) until event
- class ( win team1, win team2, draw)

Predict prop of class for each label


team1:
- wins in season til match

# Features per Team by Season per match

In [13]:
import csv
import requests
import json
import os      # for files
import csv     # csv file handling
import codecs  # because text file encoding is an issue in 2018
from datetime import datetime, date #time

In [14]:
import_file = "./db/qualifiers.csv" 
#ID;Qualifier;Values;Description
qualifiers_dict = {}
with codecs.open(import_file, 'rU','cp1252') as handle:
    reader = csv.DictReader(handle, delimiter=";", quotechar='"')
    for line in reader:
        qualifiers_dict[line["ID"]] = {'Qualifier' : line["Qualifier"], 'Values' : line['Values'],'Description' : line['Description']}
        
import_file = "./db/events.csv" 
#Event ID;Name;Description
events_dict = {}
with codecs.open(import_file, 'rU','cp1252') as handle:
    reader = csv.DictReader(handle, delimiter=";", quotechar='"')
    for line in reader:
        events_dict[line["Event ID"]] = {'Name' : line['Name'], 'Description' : line['Description']}
        
#print(qualifiers_dict)
print(qualifiers_dict['7']) # Players caught offside
print(events_dict['50'])    # Dispossessed

{'Qualifier': 'Players caught offside', 'Values': 'Players caught offside', 'Description': 'Player who was in an offside position when pass was made.'}
{'Name': 'Dispossessed', 'Description': 'Player is successfully tackled and loses possession of the ball'}


In [15]:
competitionID = 119 # AT Bundesliga
seasonID = 2018     #2018/2019
requestURLSquad = F"http://omo.akamai.opta.net/competition.php?competition={competitionID}&season_id={seasonID}&feed_type=F40&user=Hackweekend&psw=35BJ10nwf&json"
r = requests.get(requestURLSquad
                         ,timeout=None
                         ,headers={  'Content-Type': 'application/json'}
                        )    
jsondata = r.json()
docSquad = jsondata['SoccerFeed']['SoccerDocument']['Team']
matchSquad = docSquad[1]
#print(json.dumps(matchSquad, indent=4, sort_keys=True))
playerDict = {}
for team in docSquad:
    teamObj = team["@attributes"]
    teamName = teamObj['short_club_name']
    teamID = teamObj['uID'].replace('t','')
    for player in team["Player"]:
        playerID_long = player["@attributes"]["uID"]
        playerID = playerID_long.replace('p','')
        playerName = player["Name"]
        playerDict[playerID] = {'Name' : playerName, 'Team' : teamName, 'TeamID' : teamID}
#print(playerDict)

In [2]:
import platform
platform.python_version()

'3.6.6'

In [44]:
gameID = 988758
r = requests.get(F"http://omo.akamai.opta.net/?game_id={gameID}&feed_type=F9&user=Hackweekend&psw=35BJ10nwf&json"
                         ,timeout=None
                         ,headers={  'Content-Type': 'application/json'}
                        )    
jsondata = r.json()

In [45]:
matchData = jsondata['SoccerFeed']['SoccerDocument']['MatchData']
teamdata = matchData['TeamData']

- Storage for extraction
    [array of dicts]

- we need team one stats and team2 stats 

- stats can have different amount of stats 


In [51]:
onerow = {}
onerowwinner = matchData['MatchInfo']['Result']['@attributes']['Winner']
teamRefs = []
#statreflist for one time setting
statreflist= [];
i=0
while i<len(teamdata):
    teamstats = teamdata[i] 
    onerow['team_'+str(i)+'_teamRef']= teamstats['@attributes']['TeamRef']
    teamstatslist = teamstats['Stat']
    teamRefs.append(teamstats['@attributes']['TeamRef'])
    if len(statreflist)==0:
        l=0
        while l<len(teamstatslist):
            stat = teamstatslist[l]
            stattype = stat['@attributes']['Type']
            #TODO what up with fh and sh separation?            
            statvalue = stat['@value']
            onerow['team_'+str(i)+'_s_'+stattype] = float(statvalue)
            statreflist.append(stattype);
            l=l+1
    else :
        usedstats=[]
        l=0
        while l<len(teamstatslist):
            stat = teamstatslist[l]
            stattype = stat['@attributes']['Type']
            statvalue = stat['@value']
            if stattype in statreflist:
                usedstats.append(stattype)
                onerow['team_'+str(i)+'_s_'+stattype] = float(statvalue)
            l=l+1
        #anything that is in statreflist and not in usedstats needs value 0   
        openstats = list(set(statreflist).difference(set(usedstats)))
        if len(openstats)>0:
            k=0;
            while k<len(openstats):
                onerow['team_'+str(i)+'_s_'+openstats[k]] = 0
                k=k+1
    i=i+1
#who is winner team_0 or team_1
z=0
while z<len(teamRefs):
    if teamRefs[z]==onerowwinner:
        onerow['winner']='team_'+str(z)
    z=z+1
if 'winner' not in onerow:
    onerow['winner']='draw'

In [52]:
onerow

{'team_0_teamRef': 't5666',
 'team_0_s_defender_goals': 1.0,
 'team_0_s_total_flick_on': 6.0,
 'team_0_s_duel_lost': 59.0,
 'team_0_s_diving_save': 1.0,
 'team_0_s_blocked_scoring_att': 3.0,
 'team_0_s_leftside_pass': 45.0,
 'team_0_s_poss_won_att_3rd': 4.0,
 'team_0_s_dispossessed': 14.0,
 'team_0_s_accurate_cross': 1.0,
 'team_0_s_att_rf_total': 3.0,
 'team_0_s_att_goal_low_left': 1.0,
 'team_0_s_att_bx_right': 1.0,
 'team_0_s_accurate_pass': 178.0,
 'team_0_s_won_tackle': 15.0,
 'team_0_s_total_final_third_passes': 97.0,
 'team_0_s_rightside_pass': 45.0,
 'team_0_s_attempts_conceded_ibox': 10.0,
 'team_0_s_touches': 471.0,
 'team_0_s_total_fwd_zone_pass': 196.0,
 'team_0_s_att_assist_openplay': 5.0,
 'team_0_s_won_contest': 3.0,
 'team_0_s_accurate_fwd_zone_pass': 119.0,
 'team_0_s_total_chipped_pass': 21.0,
 'team_0_s_effective_head_clearance': 5.0,
 'team_0_s_lost_corners': 8.0,
 'team_0_s_fouled_final_third': 4.0,
 'team_0_s_saves': 2.0,
 'team_0_s_ontarget_scoring_att': 3.0,
 't

# input learning set

In [16]:
#get events
gameID = 988758
requestURL = F"http://omo.akamai.opta.net/?game_id={gameID}&feed_type=F24&user=Hackweekend&psw=35BJ10nwf&json"
r = requests.get(requestURL
                         ,timeout=None
                         ,headers={  'Content-Type': 'application/json'}
                        )    
jsondata = r.json()
doc = jsondata['Games']['Game']['Event']
match = doc[1]
#print(json.dumps(match, indent=4, sort_keys=True))

for each event 
- event type
- involved player
- successfull

In [17]:
for event in doc:
    ## direct attributes
    eventAttr = event['@attributes']
    minutesPlayed = int(eventAttr['min'])
    secondsPlayed = int(eventAttr['sec'])
    xCoord = float(eventAttr['x'])
    yCoord = float(eventAttr['y'])
    outcome = int(eventAttr['outcome'])
    outcomeStr = 'successful' if outcome == 1 else 'not successful' 
    
    ## event
  
    # eventType from ID
    eventTypeID = eventAttr['type_id']
    eventTypeName = events_dict[eventTypeID]['Name']
  
    # player from ID (if involved)
    playerInvolved = False
    playerName = ""
    try:
        playerID = eventAttr['player_id']
        playerObj = playerDict[playerID]
        playerName = playerObj['Name']
        teamID = playerObj['TeamID']
        teamName = playerObj['Team']
        playerInvolved = True
    except KeyError:
        playerName = 'NO PLAYER'
        playerInvolved = False
        
    uniqueEventID = eventAttr['id']
    print(F'{minutesPlayed:02}:{secondsPlayed:02} ({xCoord:005.1f}/{yCoord:005.1f}) - [{teamName}] {playerName} {eventTypeName} {outcomeStr} (Event Type ID: {eventTypeID}, UID:{uniqueEventID})')
    
    
    ## qualifiers
    try:
        qualifiers = event['Q']
    except KeyError:
        qualifiers = None
        
    if qualifiers is not None:  
        for qualifier in qualifiers:
            try:
                qualAttr = qualifier['@attributes']
                qualifierID = qualAttr['qualifier_id']
                qualifierName = qualifiers_dict[qualifierID]['Qualifier']
                qualifierValue = qualAttr['value']
                print(F'    {qualifierName} --> {qualifierValue} (ID:  {qualifierID})')
            except (TypeError, KeyError) as e:
                noQual = True
    
    

00:00 (000.0/000.0) - [TSV Hartberg] NO PLAYER Team set up successful (Event Type ID: 34, UID:1093990839)
    Resume --> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 (ID:  227)
    Team player formation --> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 0, 0, 0, 0, 0, 0 (ID:  131)
    Team formation --> 3 (ID:  130)
    Captain --> 62491 (ID:  194)
    Involved --> 98905, 150658, 77423, 164866, 62491, 83331, 92862, 62478, 119929, 160291, 73214, 165287, 162861, 427999, 19597, 98904, 429577, 165219 (ID:  30)
    Jersey number --> 31, 27, 7, 19, 26, 4, 24, 10, 34, 12, 8, 9, 11, 14, 15, 16, 17, 32 (ID:  59)
    Player position --> 1, 2, 2, 3, 2, 2, 3, 3, 4, 4, 3, 5, 5, 5, 5, 5, 5, 5 (ID:  44)
    Team kit --> 3653 (ID:  197)
00:00 (000.0/000.0) - [TSV Hartberg] NO PLAYER Team set up successful (Event Type ID: 34, UID:1436203251)
    Player position --> 1, 2, 2, 3, 2, 2, 3, 3, 4, 4, 3, 5, 5, 5, 5, 5, 5, 5 (ID:  44)
    Involved --> 208766, 43615, 62404, 113283, 42154, 223434, 43797, 170976, 

https://machinelearningmastery.com/make-predictions-scikit-learn/

In [18]:
from sklearn.feature_extraction import DictVectorizer

In [29]:
v = DictVectorizer(sparse=False)
X = v.fit_transform([onerow])

In [42]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(X)

In [43]:
transformer.transform(X)

array([[  1.44697047e-03,   3.37626444e-04,   2.41161746e-05,
          2.41161746e-05,   2.41161746e-05,   2.86982477e-03,
          9.64646982e-05,   7.23485237e-05,   5.30555840e-04,
          2.41161746e-05,   9.16414633e-04,   4.29267907e-03,
          5.06439666e-04,   3.85858793e-04,   3.37626444e-04,
          1.20580873e-04,   4.82323491e-05,   2.41161746e-05,
          2.41161746e-05,   2.41161746e-05,   2.41161746e-05,
          2.41161746e-05,   2.41161746e-05,   4.82323491e-05,
          4.82323491e-05,   2.41161746e-05,   1.44697047e-04,
          2.41161746e-05,   2.41161746e-05,   2.41161746e-05,
          2.41161746e-05,   7.23485237e-05,   4.82323491e-05,
          1.20580873e-04,   1.92929396e-04,   2.41161746e-05,
          7.23485237e-05,   2.41161746e-05,   4.82323491e-05,
          2.89394095e-04,   2.41161746e-04,   4.82323491e-05,
          1.20580873e-04,   1.20580873e-04,   8.92298458e-04,
          1.61578370e-03,   2.41161746e-05,   2.41161746e-05,
        